# **Primer análisis de datos extraidos**

#### Incluimos las librerías necesarias para analizar los datos

In [ ]:
import pandas as pd

#### Leemos los datos del JSON y lo guardamos en la variable *datos*

In [ ]:
datos = pd.read_json('traces_Articoding_Escolapias.json')

#### Usamos el metodo **head** para ver como estan estructurados los elementos del JSON

In [ ]:
datos.head(10)

#### Para empezar de forma mas sencilla analizamos el tiempo de juego de un jugador concreto, por ejemplo, de *gqoj*

* Lo primero sería extraer su primer *timestamp*  

In [ ]:
inicio = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        inicio = i[5]
        break

In [ ]:
inicio

* Ahora que ya tenemos el inicio de la partida vamos a buscar el fin de la partida, para ello nos recorremos todo el JSON y nos quedamos con el último *timestamp* del jugador

In [ ]:
fin = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        fin = i[5]

In [ ]:
fin

Parece que las trazas no son todas del mismo día, hay que buscar la forma de **filtrar por días**.

#### Ahora que ya sabemos extraer el *timestamp* de un jugador en concreto vamos a intentar recoger los de todos los jugadores

* Guardaremos los datos en un diccionario cuya clave será el **id** del jugador y su valor una pareja con el *timestamp* de inicio y de fin

In [ ]:
tiempos = dict()
for i in datos.to_numpy():
    idUser = i[0]["name"]
    timeStamp = i[5]
    
    if(idUser in tiempos):
        timeStampIni = tiempos[idUser][0]
        tiempos[idUser] = (timeStampIni, timeStamp)
    else:
        tiempos[idUser] = (timeStamp, "")

In [ ]:
pd.DataFrame(tiempos)

* Parece que ya tenemos los datos que queríamos de cada jugador, asumimos que son los correctos porque los tiempos del jugador *gqoj* coinciden con los que hemos sacado antes

* Ahora toca hacer la resta para ver el tiempo de juego de los jugadores 